In [1]:
import tensorflow as tf

In [2]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(1, activation='linear', input_dim=784))
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.1),loss='mean_squared_error',
                 metrics=['mae'])
    return model

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32')/255
x_test = x_test.reshape(10000, 784).astype('float32')/255

In [8]:
import datetime
class MyCustomCallback(tf.keras.callbacks.Callback):
    def on_train_batch_begin(self, batch, logs=None):
        print('Training: batch {} begins as {}'.format(batch, datetime.datetime.now().time()))
    def on_train_batch_end(self, batch, logs=None):
        print('Training: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

    def on_train_batch_middle(self, batch, logs=None):
        print('kok')
    
    def on_test_batch_begin(self, batch, logs=None):
        print('Evaluating: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))
    def on_test_batch_end(self, batch, logs=None):
        print('Evaluating: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

In [9]:
model = get_model()
_ = model.fit(x_train, y_train,
             batch_size=64,
             epochs=1,
             steps_per_epoch=5,
             verbose=0,
             callbacks=[MyCustomCallback()])

Training: batch 0 begins as 09:50:56.363107
Training: batch 0 ends at 09:50:56.700773
Training: batch 1 begins as 09:50:56.701020
Training: batch 1 ends at 09:50:56.702241
Training: batch 2 begins as 09:50:56.702416
Training: batch 2 ends at 09:50:56.703269
Training: batch 3 begins as 09:50:56.703429
Training: batch 3 ends at 09:50:56.704256
Training: batch 4 begins as 09:50:56.704441
Training: batch 4 ends at 09:50:56.705456


In [12]:
_ = model.evaluate(x_test, y_test, batch_size=128, verbose=0, steps=5, callbacks=[MyCustomCallback()])

Evaluating: batch 0 begins at 09:52:49.584187
Evaluating: batch 0 ends at 09:52:49.586923
Evaluating: batch 1 begins at 09:52:49.587363
Evaluating: batch 1 ends at 09:52:49.588301
Evaluating: batch 2 begins at 09:52:49.588466
Evaluating: batch 2 ends at 09:52:49.589385
Evaluating: batch 3 begins at 09:52:49.589592
Evaluating: batch 3 ends at 09:52:49.590696
Evaluating: batch 4 begins at 09:52:49.590858
Evaluating: batch 4 ends at 09:52:49.591778


In [13]:
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):

  def on_train_batch_end(self, batch, logs=None):
    print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

  def on_test_batch_end(self, batch, logs=None):
    print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

  def on_epoch_end(self, epoch, logs=None):
    print('The average loss for epoch {} is {:7.2f} and mean absolute error is {:7.2f}.'.format(epoch, logs['loss'], logs['mae']))

model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=3,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback()])

For batch 0, loss is   26.59.
For batch 1, loss is 1007.81.
For batch 2, loss is   29.20.
For batch 3, loss is    7.61.
For batch 4, loss is    6.90.
The average loss for epoch 0 is  215.62 and mean absolute error is    8.53.
For batch 0, loss is    9.09.
For batch 1, loss is    7.00.
For batch 2, loss is    5.86.
For batch 3, loss is    7.03.
For batch 4, loss is    5.00.
The average loss for epoch 1 is    6.80 and mean absolute error is    2.17.
For batch 0, loss is    4.93.
For batch 1, loss is    4.81.
For batch 2, loss is    4.65.
For batch 3, loss is    3.96.
For batch 4, loss is    4.39.
The average loss for epoch 2 is    4.55 and mean absolute error is    1.71.


In [15]:
_ = model.evaluate(x_test, y_test, batch_size=128, verbose=0, steps=20,
          callbacks=[LossAndErrorPrintingCallback()])

For batch 0, loss is    9.10.
For batch 1, loss is    9.27.
For batch 2, loss is    8.56.
For batch 3, loss is    9.04.
For batch 4, loss is    9.86.
For batch 5, loss is    9.86.
For batch 6, loss is    8.73.
For batch 7, loss is    8.39.
For batch 8, loss is    9.34.
For batch 9, loss is   10.57.
For batch 10, loss is    8.20.
For batch 11, loss is    9.11.
For batch 12, loss is   10.03.
For batch 13, loss is   10.05.
For batch 14, loss is    8.35.
For batch 15, loss is    7.75.
For batch 16, loss is   10.29.
For batch 17, loss is    8.54.
For batch 18, loss is   10.36.
For batch 19, loss is    9.37.


In [16]:
import numpy as np

class EarlyStoppingAtMinLoss(tf.keras.callbacks.Callback):
  """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """

  def __init__(self, patience=0):
    super(EarlyStoppingAtMinLoss, self).__init__()

    self.patience = patience

    # best_weights to store the weights at which the minimum loss occurs.
    self.best_weights = None

  def on_train_begin(self, logs=None):
    # The number of epoch it has waited when loss is no longer minimum.
    self.wait = 0
    # The epoch the training stops at.
    self.stopped_epoch = 0
    # Initialize the best as infinity.
    self.best = np.Inf

  def on_epoch_end(self, epoch, logs=None):
    current = logs.get('loss')
    if np.less(current, self.best):
      self.best = current
      self.wait = 0
      # Record the best weights if current results is better (less).
      self.best_weights = self.model.get_weights()
    else:
      self.wait += 1
      if self.wait >= self.patience:
        self.stopped_epoch = epoch
        self.model.stop_training = True
        print('Restoring model weights from the end of the best epoch.')
        self.model.set_weights(self.best_weights)

  def on_train_end(self, logs=None):
    if self.class LearningRateScheduler(tf.keras.callbacks.Callback):
  """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

  def __init__(self, schedule):
    super(LearningRateScheduler, self).__init__()
    self.schedule = schedule

  def on_epoch_begin(self, epoch, logs=None):
    if not hasattr(self.model.optimizer, 'lr'):
      raise ValueError('Optimizer must have a "lr" attribute.')
    # Get the current learning rate from model's optimizer.
    lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
    # Call schedule function to get the scheduled learning rate.
    scheduled_lr = self.schedule(epoch, lr)
    # Set the value back to the optimizer before this epoch starts
    tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
    print('\nEpoch %05d: Learning rate is %6.4f.' % (epoch, scheduled_lr)) > 0:
      print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))

In [17]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=30,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss()])

For batch 0, loss is   31.97.
For batch 1, loss is 1163.91.
For batch 2, loss is   28.82.
For batch 3, loss is   10.51.
For batch 4, loss is    7.59.
The average loss for epoch 0 is  248.56 and mean absolute error is    9.27.
For batch 0, loss is    7.60.
For batch 1, loss is    6.13.
For batch 2, loss is    4.81.
For batch 3, loss is    4.72.
For batch 4, loss is    6.06.
The average loss for epoch 1 is    5.86 and mean absolute error is    2.02.
For batch 0, loss is    4.89.
For batch 1, loss is    4.74.
For batch 2, loss is    5.29.
For batch 3, loss is    5.03.
For batch 4, loss is    5.50.
The average loss for epoch 2 is    5.09 and mean absolute error is    1.83.
For batch 0, loss is    5.18.
For batch 1, loss is    4.31.
For batch 2, loss is    5.17.
For batch 3, loss is    8.85.
For batch 4, loss is   10.40.
The average loss for epoch 3 is    6.78 and mean absolute error is    2.08.
Restoring model weights from the end of the best epoch.
Epoch 00004: early stopping


In [18]:
class LearningRateScheduler(tf.keras.callbacks.Callback):
  """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

  def __init__(self, schedule):
    super(LearningRateScheduler, self).__init__()
    self.schedule = schedule

  def on_epoch_begin(self, epoch, logs=None):
    if not hasattr(self.model.optimizer, 'lr'):
      raise ValueError('Optimizer must have a "lr" attribute.')
    # Get the current learning rate from model's optimizer.
    lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
    # Call schedule function to get the scheduled learning rate.
    scheduled_lr = self.schedule(epoch, lr)
    # Set the value back to the optimizer before this epoch starts
    tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
    print('\nEpoch %05d: Learning rate is %6.4f.' % (epoch, scheduled_lr))
    
LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (3, 0.05), (6, 0.01), (9, 0.005), (12, 0.001)
]

def lr_schedule(epoch, lr):
  """Helper function to retrieve the scheduled learning rate based on epoch."""
  if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
    return lr
  for i in range(len(LR_SCHEDULE)):
    if epoch == LR_SCHEDULE[i][0]:
      return LR_SCHEDULE[i][1]
  return lr

model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=15,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(), LearningRateScheduler(lr_schedule)])


Epoch 00000: Learning rate is 0.1000.
For batch 0, loss is   31.24.
For batch 1, loss is  935.79.
For batch 2, loss is   31.94.
For batch 3, loss is    7.09.
For batch 4, loss is    7.67.
The average loss for epoch 0 is  202.75 and mean absolute error is    8.51.

Epoch 00001: Learning rate is 0.1000.
For batch 0, loss is    3.73.
For batch 1, loss is    7.76.
For batch 2, loss is    5.91.
For batch 3, loss is    7.50.
For batch 4, loss is    6.93.
The average loss for epoch 1 is    6.37 and mean absolute error is    2.09.

Epoch 00002: Learning rate is 0.1000.
For batch 0, loss is    4.45.
For batch 1, loss is    6.24.
For batch 2, loss is    6.32.
For batch 3, loss is    6.48.
For batch 4, loss is    6.77.
The average loss for epoch 2 is    6.05 and mean absolute error is    1.95.

Epoch 00003: Learning rate is 0.0500.
For batch 0, loss is    5.95.
For batch 1, loss is    3.51.
For batch 2, loss is    3.82.
For batch 3, loss is    4.83.
For batch 4, loss is    3.27.
The average loss